In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
from huggingface_hub import login, upload_file
from datasets import load_dataset
from pathlib import Path
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib

In [ ]:
# Setup and Login
load_dotenv()
hf_token = os.getenv("HF_TOKEN")
hub_model_id = os.getenv("HUB_MODEL_ID")
login(token=hf_token)

In [ ]:
# Load dataset
project_root = Path().cwd().resolve().parent
dataset_dir = project_root / "data" / "processed"
dataset = load_dataset('csv', data_files={'train': str(dataset_dir / 'train.csv'), 'val': str(dataset_dir / 'val.csv'), 'test': str(dataset_dir / 'test.csv')})
dataset

In [ ]:
# Train embeddings generation
# Load the all-MiniLM-L6-v2 model, optimized for semantic similarity
model_name_st = "sentence-transformers/all-MiniLM-L6-v2"
st_model = SentenceTransformer(model_name_st)

# Extract sentences and labels
train_utterances = list(dataset['train']['user_utterance'])
train_labels = list(dataset['train']['label'])

# Convert sentences to numerical vectors (embeddings)
train_embeddings = st_model.encode(train_utterances, show_progress_bar=True)

print(f"Created {len(train_embeddings)} embeddings, each of dimension {train_embeddings.shape[1]}")

In [ ]:
from sklearn.manifold import TSNE

# Visualization of Embeddings with t-SNE
# 1. Take a sample of data to speed up computation
# t-SNE is computationally intensive. 10,000 points is a good compromise.
num_samples = 10000
if len(train_labels) > num_samples:
	print(f"Using a random sample of {num_samples} points for visualization.")
	indices = np.random.choice(len(train_labels), num_samples, replace=False)
	sampled_embeddings = np.array(train_embeddings)[indices]
	sampled_labels = np.array(train_labels)[indices]
else:
	sampled_embeddings = np.array(train_embeddings)
	sampled_labels = np.array(train_labels)

# 2. Reduce dimensionality from 384 to 2 using t-SNE
print("Running t-SNE... (this may take a few minutes)")
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
embeddings_2d = tsne.fit_transform(sampled_embeddings)

# 3. Create the scatter plot
print("Creating the plot...")
plt.figure(figsize=(14, 10))

# Use Seaborn for a nicer plot with automatic legend
sns.scatterplot(
	x=embeddings_2d[:, 0],
	y=embeddings_2d[:, 1],
	hue=sampled_labels,
	palette=sns.color_palette("hsv", len(np.unique(sampled_labels))),
	s=50, # Point size
	alpha=0.7 # Transparency
)

# Add titles and move the legend outside the plot
plt.title('Visualization of Embeddings with t-SNE', fontsize=16)
plt.xlabel('Dimension 1 (t-SNE)')
plt.ylabel('Dimension 2 (t-SNE)')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., title='Intents')
plt.grid(True)
plt.show()


In [ ]:
# Addestramento del classificatore KNN
print("Addestramento del classificatore KNN...")
# Inizializziamo il classificatore
# n_neighbors=5 è un valore di default comune
knn = KNeighborsClassifier(n_neighbors=5, metric='cosine')

# Addestriamo il KNN usando gli embeddings e le etichette
knn.fit(train_embeddings, train_labels)

print("Addestramento completato.")

In [ ]:
# Test embeddings generation
# Extract sentences and labels
test_utterances = list(dataset['test']['user_utterance'])
test_labels = list(dataset['test']['label'])

# Convert sentences to numerical vectors (embeddings)
test_embeddings = st_model.encode(test_utterances, show_progress_bar=True)

In [ ]:
# Get predictions from the KNN model
test_predictions = knn.predict(test_embeddings)

In [ ]:
# Classification Report
target_names = sorted(set(dataset['test']['new_intent']))
report = classification_report(test_labels, test_predictions, target_names=target_names, digits=4)
print("\nClassification Report:")
print(report)

In [ ]:
# Confusion Matrix
cm = confusion_matrix(test_labels, test_predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Model Saving
model_dir = project_root / "models"
knn_model_path = model_dir / "intent_classifier" / "knn_model.joblib"
joblib.dump(knn, knn_model_path)